In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [30]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-31388")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Machine Learning Engineer – AI/ML (Remote Work Option)

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsData & AnalyticsMachine Learning Engineer – AI/ML (Remote Work Option)Be

In [31]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [32]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Machine Learning Engineer – AI/ML',
  'experience': 'Bachelor’s degree in a relevant field, like computer science or software engineering, or equivalent experience.',
  'skills': ['Understanding of Machine Learning, its applications, and the lifecycle of an ML application in production',
   'Ability to meaningfully communicate, written, orally, and visually technical topics with peers',
   'Experience working in and/or collaborating with a partial or fully distributed team',
   'Understanding of data structures, algorithms, and data solutions',
   'Experience in applying Python (or another language commonly used in the field of ML, such as Scala, Julia, C++) and SQL to ML and/or software and data engineering tasks',
   'Familiarity with ETL, ML, or analytics technologies such as Scikit-learn, Pandas, OpenCV, NumPy, TensorFlow, or similar platforms and frameworks',
   'Awareness of data science platforms (like Databricks or SageMaker), distributed engines (like Spark and AWS 

In [33]:
type(json_res)

list

In [34]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular, .NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
...,...,...
100,"Cloud, Google Cloud, Kubernetes",https://example.com/gcp-kubernetes-portfolio
101,"AI, Transformer Models, Python",https://example.com/transformer-ai-portfolio
102,"Web Development, Bootstrap, jQuery",https://example.com/bootstrap-jquery-portfolio
103,"Frontend, Vue.js, Vuetify",https://example.com/vuejs-vuetify-portfolio


In [35]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [36]:
job = json_res[0]
job['skills']

['Understanding of Machine Learning, its applications, and the lifecycle of an ML application in production',
 'Ability to meaningfully communicate, written, orally, and visually technical topics with peers',
 'Experience working in and/or collaborating with a partial or fully distributed team',
 'Understanding of data structures, algorithms, and data solutions',
 'Experience in applying Python (or another language commonly used in the field of ML, such as Scala, Julia, C++) and SQL to ML and/or software and data engineering tasks',
 'Familiarity with ETL, ML, or analytics technologies such as Scikit-learn, Pandas, OpenCV, NumPy, TensorFlow, or similar platforms and frameworks',
 'Awareness of data science platforms (like Databricks or SageMaker), distributed engines (like Spark and AWS cloud), and CI/CD pipelines and containerization']

In [37]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}]]

In [38]:
job

{'role': 'Machine Learning Engineer – AI/ML',
 'experience': 'Bachelor’s degree in a relevant field, like computer science or software engineering, or equivalent experience.',
 'skills': ['Understanding of Machine Learning, its applications, and the lifecycle of an ML application in production',
  'Ability to meaningfully communicate, written, orally, and visually technical topics with peers',
  'Experience working in and/or collaborating with a partial or fully distributed team',
  'Understanding of data structures, algorithms, and data solutions',
  'Experience in applying Python (or another language commonly used in the field of ML, such as Scala, Julia, C++) and SQL to ML and/or software and data engineering tasks',
  'Familiarity with ETL, ML, or analytics technologies such as Scikit-learn, Pandas, OpenCV, NumPy, TensorFlow, or similar platforms and frameworks',
  'Awareness of data science platforms (like Databricks or SageMaker), distributed engines (like Spark and AWS cloud), a

In [29]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are Muhammad, a business development executive at Nexify Solutions. Nexify Solutions is a pioneering tech consultancy 
    specializing in innovative software solutions and AI-driven automation. 
    We excel in delivering customized digital transformation strategies that enhance operational efficiency, 
    drive scalability, and reduce costs for businesses across various industries. 
    Your task is to write a cold email to the client regarding the job mentioned above, emphasizing Nexify Solutions' 
    capability to meet their needs.
    Also, include the most relevant examples from the following links to highlight Nexify's portfolio: {link_list}
    Remember you are Kavita, BDE at Nexify Solutions.
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    
    """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)


Subject: Expert Machine Learning Engineers for Your AI/ML Team

Dear Hiring Manager,

I came across your job posting for a Machine Learning Engineer – AI/ML, and I'm excited to introduce you to AtliQ, a leading AI & Software Consulting company. Our team of experts has a proven track record of delivering tailored solutions that drive business growth through automation.

With a strong background in developing advanced analytics systems, our Machine Learning Engineers can help you build and deploy AI/ML applications that directly impact your business. Our expertise includes:

* Developing and deploying machine learning models using Python, Scikit-learn, Pandas, OpenCV, NumPy, and TensorFlow
* Designing and implementing data structures, algorithms, and data solutions
* Collaborating with cross-disciplinary teams to enable data-driven decision making
* Working with ETL, ML, or analytics technologies and data science platforms like Databricks or SageMaker
* Implementing CI/CD pipelines and c